In [1]:
import pandas as pd
import numpy as np

f = pd.read_csv('review_final.csv')
keyword_list = ['sick', 'dirty', 'disgusting', 'vomit', 'threw up', 'food poison', 'diarrhea', 'filthy', 'gross', 'throw up', 'throwing up', 'indigestion', 'nausea', 'nauseous', 'upset stomach', 'the runs', ' bug ', ' rat ', 'mice', 'mouse', 'expired', 'listeria', 'contaminate', 'bacteria', 'salmonella', 'e coli', 'infest', 'retch', ' gag ', 'stench', 'e-coli', 'tainted']
data = {"word_count": 0,
    "flagged_reviews": 0,
    "total_reviews": 0,
    "one_star_count": 0,
    "total_stars": 0}
# Clean NaN's from df
f[['text']] = f[['text']].fillna(value=' ')
f = f.fillna(value=0)

In [2]:
f.head()

,id,business_id,user_id,stars,date,text,useful,funny,cool
0,---94vtJ_5o_nikEs6hUjg,R1PQEK6qvrZVC9qcWfKvDA,c2MQ_LPuvtiiKFR_-OY9pg,5,09/17/2014 00:00:00,One of my absolute favorite restaurants! I usu...,1.0,0.0,0.0
1,---WDP9kwKyVQiw9GTgNmQ,3kdSl5mo9dWC4clrQjEDGg,tRXe5HRTDsUNqL3yNSquMw,1,03/17/2014 00:00:00,2Nd time eating here today.1st time was great ...,0.0,0.0,0.0
2,---zHMCae68gIbSbtXxD5w,4RoTEeqB_MNn6yaqZmlZHg,rypcWiSNGM0suWsiSLh9xA,4,08/29/2015 00:00:00,From 8/22/15 \n\nThe Regal 18 is located in Vi...,3.0,1.0,1.0
3,--03fUVGimHb46r8XKjn2A,TTDMJetAQKfxVzKZy4Z_2Q,3q6AU-YWv_ORyZaxelasTg,1,06/27/2017 00:00:00,Allegiant is a disaster. Their fares are chea...,1.0,0.0,0.0
4,--0cKK_pJr0I9lk82aeQcQ,WYw3Uf56DT5IwpaLNnCH5Q,Vkf4e4SLFm3gApONFnWk7Q,4,12/03/2013 00:00:00,I booked a deluxe suite in vdara for this past...,1.0,0.0,0.0


In [3]:
# new df with unique business_id as key
biz = f.business_id.unique()
df = pd.DataFrame(data, index = biz)
df.describe()

,flagged_reviews,one_star_count,total_reviews,total_stars,word_count
count,9772.0,9772.0,9772.0,9772.0,9772.0
mean,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0


In [4]:
def count_keywords(review):
    c = 0
    for word in keyword_list:
        wc = review.count(word)
        c = c + wc
    return c

In [5]:
# get aggregate flagged 
df = pd.DataFrame(data, index = biz)
for i in range(len(f)):
    bid = f["business_id"][i]
    review = f["text"][i]
    useful = int(f["useful"][i])
    stars = int(f["stars"][i])
    wc = df["word_count"][bid]
    fr = df["flagged_reviews"][bid]
    tr = df["total_reviews"][bid]
    os = df["one_star_count"][bid]
    ts = df["total_stars"][bid]
    nwc = count_keywords(review)
    # update if any flagged words
    if (nwc > 0):
        # note: useful votes multiply new word count and flagged review count
        df.at[bid, "word_count"] = wc + nwc * (useful + 1)
        df.at[bid, "flagged_reviews"] = fr + 1 + useful
    # update if one star review
    if (stars == 1):
        df.at[bid, "one_star_count"] = os + 1
    # always update total_review_count & total_stars
    df.at[bid, "total_reviews"] = tr + 1
    df.at[bid, "total_stars"] = ts + stars
    # monitor progress
    if (i%20000 == 0 or i == len(f)-1):
        print("{:.2%}".format(i/len(f)))

0.00%
2.47%
4.94%
7.42%
9.89%
12.36%
14.83%
17.30%
19.78%
22.25%
24.72%
27.19%
29.66%
32.14%
34.61%
37.08%
39.55%
42.02%
44.50%
46.97%
49.44%
51.91%
54.38%
56.86%
59.33%
61.80%
64.27%
66.74%
69.22%
71.69%
74.16%
76.63%
79.11%
81.58%
84.05%
86.52%
88.99%
91.47%
93.94%
96.41%
98.88%
100.00%


In [6]:
# find means & proportions
df["proportion_flagged_reviews"] = df["flagged_reviews"]/df["total_reviews"]
df["average_flagged_words_per_review"] = df["word_count"]/df["total_reviews"]
df["average_stars_per_review"] = df["total_stars"]/df["total_reviews"]
df["proportion_one_star"] = df["one_star_count"]/df["total_reviews"]

In [7]:
df.head()

,flagged_reviews,one_star_count,total_reviews,total_stars,word_count,proportion_flagged_reviews,average_flagged_words_per_review,average_stars_per_review,proportion_one_star
R1PQEK6qvrZVC9qcWfKvDA,30,7,151,647,43,0.198675,0.284768,4.284768,0.046358
3kdSl5mo9dWC4clrQjEDGg,91,46,2594,11525,100,0.035081,0.038551,4.442945,0.017733
4RoTEeqB_MNn6yaqZmlZHg,27,14,158,560,36,0.170886,0.227848,3.544304,0.088608
TTDMJetAQKfxVzKZy4Z_2Q,287,473,639,1010,339,0.449139,0.530516,1.580595,0.740219
WYw3Uf56DT5IwpaLNnCH5Q,453,197,2317,8887,582,0.195511,0.251187,3.835563,0.085024


In [8]:
print("Limitation:\nExample of false positive for nice restaurant: \"...it's 100% worth it, stick to McDonald's if you're on a budget and want diarrhea.\"")

Limitation:
Example of false positive for nice restaurant: "...it's 100% worth it, stick to McDonald's if you're on a budget and want diarrhea."


In [12]:
# load current businesses data
b = pd.read_csv('business_final.csv')
b.head()

,Column,business_id,name,neighborhood,address,postal_code,latitude,longitude,stars,review_count,categories
0,6,kCoE3jvEtg6UVz5SOD3GVw,BDJ Realty,Summerlin,2620 Regatta Dr,89128,36.2074,-115.268460,4.0,5,Real Estate Services;Real Estate;Home Services...
1,26,VBHEsoXQb2AQ76J9l8h1uQ,Alfredo's Jewelry,Southeast,5775 S Eastern,89119,36.0851,-115.119421,4.5,23,Shopping;Jewelry;Watch Repair;Local Services
2,49,_F3AMoo_zdl-he384ISQbw,Rock of Ages,NaN,3700 W Flamingo Rd,89109,36.1170,-115.185696,4.0,213,Arts & Entertainment;Performing Arts
3,53,4srfPk1s8nlm1YusyDUbjg,Subway,Southeast,6889 S Eastern Ave,89119,36.0647,-115.118954,2.5,6,Fast Food;Restaurants;Sandwiches
4,60,v2GJWvZqEAjUc22hZUYzYw,John Armond Actor's Studio,Westside,8125 W Sahara Ave,89117,36.1425,-115.268731,4.5,12,Education;Performing Arts;Specialty Schools;Ar...


In [14]:
# bring values from business_final into new df
for i in range(len(b)):
    bid = b["business_id"][i]
    df.at[bid, "name"] = b["name"][i]
    df.at[bid, "neighborhood"] = b["neighborhood"][i]
    df.at[bid, "address"] = b["address"][i]
    df.at[bid, "postal_code"] = b["postal_code"][i]
    df.at[bid, "stars"] = b["stars"][i]
    df.at[bid, "review_count"] = b["review_count"][i]
    df.at[bid, "categories"] = b["categories"][i]

In [15]:
df.head()

,flagged_reviews,one_star_count,total_reviews,total_stars,word_count,proportion_flagged_reviews,average_flagged_words_per_review,average_stars_per_review,proportion_one_star,name,neighborhood,address,postal_code,stars,review_count,categories
R1PQEK6qvrZVC9qcWfKvDA,30,7,151,647,43,0.198675,0.284768,4.284768,0.046358,Braddah's Island Style,University,4632 S Maryland Pkwy,89119.0,4.5,151.0,Food;Restaurants;Hawaiian;Desserts;Mexican;Tex...
3kdSl5mo9dWC4clrQjEDGg,91,46,2594,11525,100,0.035081,0.038551,4.442945,0.017733,Egg & I,Westside,4533 W Sahara Ave,89102.0,4.5,2595.0,Restaurants;Burgers;American (Traditional);San...
4RoTEeqB_MNn6yaqZmlZHg,27,14,158,560,36,0.170886,0.227848,3.544304,0.088608,Regal Cinemas Village Square 18,Westside,9400 W Sahara Ave,89117.0,3.5,158.0,Cinema;Arts & Entertainment
TTDMJetAQKfxVzKZy4Z_2Q,287,473,639,1010,339,0.449139,0.530516,1.580595,0.740219,Allegiant Airlines,Southeast,5757 Wayne Newton Blvd,89119.0,1.5,639.0,Airlines;Airports;Transportation;Hotels & Travel
WYw3Uf56DT5IwpaLNnCH5Q,453,197,2317,8887,582,0.195511,0.251187,3.835563,0.085024,Vdara Hotel,The Strip,2600 W Harmon Ave,89158.0,4.0,2315.0,Beauty & Spas;Hotels;Resorts;Hotels & Travel;E...


In [16]:
df.to_csv("aggregate_missing_inspections.csv")